In [14]:
from pyshex import ShExEvaluator
from rdflib import Namespace

In [15]:
shex = """
BASE <http://example.org/ex/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX ex: <http://ex.example/#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX : <http://hl7.org/fhir/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX loinc: <http://loinc.org/rdf#>
PREFIX snomed: <http://www.ihtsdo.org/>

start = @<BloodPressureMeasurementShape>

<PatientShape> {                    # A Patient has:
:name xsd:string*;                        #   one or more names
:birthdate xsd:date?   ;          #   and an optional birthdate.
}

<BloodPressureMeasurementShape>  {

rdfs:label  xsd:string ;
:subject @<PatientShape> ;
:hasmeasurementDate  @<BPDateShape> ;
:valueSBP @<SBPvalueShape> ;
:valueDBP @<DBPvalueShape> ;
:valueABP @<ABPvalueShape>? ;
:isAffectedBy @<BodyPositionShape>? ;
((:hasMethod @<BPMeasurementInvasiveMethodShape> ;
 :hasLocation IRI{0} ;
 :hasType IRI{0} ;
 :hasCuffSize IRI{0}
)?
|
(:hasMethod @<BPMeasurementNonInvasiveMethodShape>  ;
| :hasLocation @<BPMeasurementLocationShape> ;
| :hasType @<DEPShape> ;
| :hasCuffSize @<BPCuffSizeShape> 
)*
)
}


<SBPvalueShape> {
 :CodeableConcept.coding {
       a [loinc:8480-6] ;  #Systolic BP
  } ;
 :QuantityCharacteristics {
 :Quantity.value {:value xsd:decimal} ;
 :Quantity.unit {:value [<mmHg>]} ;
 }
}

<DBPvalueShape>  {
 :CodeableConcept.coding {
       a [loinc:8462-4] ;  #Diastolic BP
  } ;
 :QuantityCharacteristics {
 :Quantity.value {:value xsd:decimal} ;
 :Quantity.unit {:value [<mmHg>]} ;
 }
}

<ABPvalueShape> {
   :CodeableConcept.coding {
       a [loinc:8478-0] ;  #Average BP
  } ;
 :QuantityCharacteristics {
 :Quantity.value {:value xsd:decimal} ;
 :Quantity.unit {:value [<mmHg>]} ;
 }
}

<BPMeasurementMethodShape>  {
:CodeableConcept.coding {
       a [loinc:8357-6] ;  #BP measurement method
 } ;
 :QuantityCharacteristics {
 :Quantity.value {:value [<invasive> <non-invasive>]} ;
 }
}

<BPMeasurementInvasiveMethodShape>  @<BPMeasurementMethodShape>  AND {
 :QuantityCharacteristics {
 :Quantity.value {:value [<invasive>]} ;
 }
} 

<BPMeasurementNonInvasiveMethodShape> @<BPMeasurementMethodShape> AND {
 :QuantityCharacteristics {
 :Quantity.value {:value [<non-invasive>]} ;
 }
} 

<BPDateShape> {
   :CodeableConcept.coding {
       a [loinc:64991-3] ;  #Date of Observation
  } ; 
 :QuantityCharacteristics {
 :Quantity.value {:value xsd:dateTime} ;
 }
}

<BPMeasurementLocationShape> {
 :CodeableConcept.coding {
       a [loinc:41904-4] ;  #Blood pressure measurement site
  } ; 
 :QuantityCharacteristics {
 :Quantity.value {:value [<UpperArmStructure> <RightUpperArmStructure> <LeftUpperArmStructure> <ThighStructure> <StructureOfRightThigh> <StructureOfLeftThigh> <WristRegionStructure> <StructureOfRightWrist> <StructureOfLeftWrist> <FingerStructure> <AnkleRegionStructure> <StructureOfRightAnkle> <StructureOfLeftAnkle>]} ;
 }
}

<DEPShape> {
   :CodeableConcept.coding {
       a [snomed:85549003] ;  #Diastolic End Point
  } ; 
 :QuantityCharacteristics {
 :Quantity.value {:value [<typeIV> <typeV>]} ;
 }
}

<BodyPositionShape> {
 :CodeableConcept.coding {
       a [loinc:8361-8] ;  #Body Position
  } ; 
 :QuantityCharacteristics {
 :Quantity.value {:value [<sittingposition> <recumbentbodyposition> <orthostaticbodyposition> <positionwithtilt> <trendelenburgposition>]} ;
 }
}

<BPCuffSizeShape> {
 :CodeableConcept.coding {
       a [loinc:8358-4] ;  #BP Cuff Size
  } ; 
 :QuantityCharacteristics {
 :Quantity.value {:value [<AdultStandard> <AdultSmall> <AdultLarge> <AdultThighSize> <ChildSize> <YoungChildSize> <NeonateSize>]} ;
 }
}
"""

In [34]:
rdf = """
BASE <http://example.org/ex/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX ex: <http://ex.example/#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX : <http://hl7.org/fhir/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX loinc: <http://loinc.org/rdf#>
PREFIX snomed: <http://www.ihtsdo.org/>

<Patient2>
  :name 'Bob' ;
  :birthdate '1999-12-31'^^xsd:date ;
  :has <BPM1> .

<BPDate1>
  :CodeableConcept.coding [
       a loinc:64991-3]; #Date of Observation
  :QuantityCharacteristics [
  :Quantity.value [ :value '2013-04-02T09:30:10+01:00'^^xsd:dateTime ] ;
 ].

<SBP1>
  :CodeableConcept.coding [
       a loinc:8480-6]; #Systolic BP
  :QuantityCharacteristics [
  :Quantity.value [ :value '140'^^xsd:decimal ] ;
  :Quantity.unit [ :value <mmHg> ] ;
 ].


<DBP1>
  :CodeableConcept.coding [
       a loinc:8462-4;    ]; #Diastolic BP
  :QuantityCharacteristics [
  :Quantity.value [ :value '90'^^xsd:decimal ] ;
  :Quantity.unit [ :value <mmHg> ] ;
 ].

<ABP1>
  :CodeableConcept.coding [
       a loinc:8478-0]; #Average BP
  :QuantityCharacteristics [
  :Quantity.value [ :value '97'^^xsd:decimal ] ;
  :Quantity.unit [ :value <mmHg> ] ;
 ].


<BPMLocation1>
  :CodeableConcept.coding [
       a loinc:41904-4]; #Blood pressure measurement site
  :QuantityCharacteristics [
  :Quantity.value [ :value <FingerStructure> ] ;
 ].

<BodyPosition1>
  :CodeableConcept.coding [
       a loinc:8361-8]; #Body Position
  :QuantityCharacteristics [
  :Quantity.value [ :value <sittingposition> ] ;
 ].

<BPCuffSize1>
  :CodeableConcept.coding [
       a loinc:8358-4]; #BP Cuff Size
  :QuantityCharacteristics [
  :Quantity.value [ :value <AdultStandard> ] ;
 ].

<DEP1>
  :CodeableConcept.coding [
       a snomed:85549003]; #Diastolic End Point
  :QuantityCharacteristics [
  :Quantity.value [ :value <typeV> ] ;
 ].

<BPMMethod1>
  :CodeableConcept.coding [
       a loinc:8357-6]; #BP measurement method
  :QuantityCharacteristics [
  :Quantity.value [ :value <invasive> ] ;
 ]. 

<BPMMethod2>
  :CodeableConcept.coding [
       a loinc:8357-6]; #BP measurement method
  :QuantityCharacteristics [
  :Quantity.value [ :value <non-invasive> ] ;
 ]. 

<BPM1>
  a :BloodPressure;
  rdfs:label 'First BP measurement' ;
  :subject  <Patient2> ;
  :hasmeasurementDate <BPDate1> ;
  :valueSBP <SBP1> ;
  :valueDBP <DBP1> ;
  :valueABP <ABP1> ;
  :hasMethod <BPMMethod1> ; 
  :hasLocation <BPMLocation1> ;
  :hasType <DEP1> ;
  :hasCuffSize <BPCuffSize1> ;
  :isAffectedBy <BodyPosition1> .
"""

In [32]:
focus = "http://example.org/ex/BPM1"

In [33]:
results = ShExEvaluator().evaluate(rdf, shex, focus)
for r in results:
    if r.result:
        print("PASS")
    else:
        print(f"FAIL: {r.reason}")

PASS
